In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [2]:
# Example Bloom Filter

from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = double_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [3]:
# Example Bloom Filter 2

from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = double_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
h = double_sha256(b'goodbye')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [4]:
# Example Bloom Filter 3

from helper import double_sha256, hash160

bit_field_size = 10
bit_field = [0] * bit_field_size

phrase1 = b'hello world'
h1 = double_sha256(phrase1)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase1)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
phrase2 = b'goodbye'
h1 = double_sha256(phrase2)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase2)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [5]:
# Example BIP0037 Bloom Filter

from helper import murmur3
from bloomfilter import BIP37_CONSTANT

field_size = 2
num_functions = 2
tweak = 42

bit_field_size = field_size * 8
bit_field = [0] * bit_field_size

for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Exercise 1

#### 1.1 Given a Bloom Filter with these parameters: size=10, function count=5, tweak=99, which bits are set after adding these items? 

 * `b'Hello World'`
 * `b'Goodbye!'`

#### 1.2. Make [this test](/edit/session8/bloomfilter.py) pass.

In [6]:
# Exercise 1.1
from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')

# bit_field_size is 8 * field_size
bit_field_size = field_size * 8
# create a bit field with the appropriate size
bit_field = [0] * bit_field_size

# for each item you want to add to the filter
for item in items:
    # iterate function_count number of times
    for i in range(function_count):
        # BIP0037 spec seed is i*BIP37_CONSTANT + tweak
        seed = i * BIP37_CONSTANT + tweak
        # get the murmur3 hash given that seed
        h = murmur3(item, seed=seed)
        # set the bit to be h mod the bit_field_size
        bit = h % bit_field_size
        # set the bit_field at the index bit to be 1
        bit_field[bit] = 1
# print the bit field converted to bytes using bit_field_to_bytes in hex
print(bit_field_to_bytes(bit_field).hex())

4000600a080000010940


In [7]:
# Exercise 1.2

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_add'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 2

#### 2.1. Make [this test](/edit/session8/bloomfilter.py) pass.

#### 2.2. Do the following:

* Connect to a testnet node
* Load a filter for your testnet address
* Send a request for transactions from the block which had your previous testnet transaction
* Receive the merkleblock and tx messages.

In [8]:
# Exercise 2.1

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_filterload'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [9]:
# Exercise 2.2

from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import decode_base58, double_sha256, little_endian_to_int
from merkleblock import MerkleBlock
from network import (
    FILTERED_BLOCK_DATA_TYPE, 
    GetDataMessage,
    SimpleNode,
)

block_hash = bytes.fromhex('00000000000001fd16e986436252c023b2f9ba729319309189af9ab5be9d4ff9')
passphrase = b'jimmy@programmingblockchain.com secret passphrase'  # FILL THIS IN
secret = little_endian_to_int(double_sha256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
filter_size = 30
filter_num_functions = 5
filter_tweak = 90210  # FILL THIS IN

# get the hash160 of the address, which is what we need to add to our filter
h160 = decode_base58(addr)
# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
bf = BloomFilter(filter_size, filter_num_functions, filter_tweak)
# add the h160 to the bloom filter
bf.add(h160)

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)
# complete the handshake
node.handshake()
# load the bloom filter with the filterload command
node.send(b'filterload', bf.filterload())
# create a getdata message
getdata = GetDataMessage()
# add a FILTERED_BLOCK_DATA_TYPE for the block hash above
getdata.add_data(FILTERED_BLOCK_DATA_TYPE, block_hash)
# send a getdata message for a FILTERED_BLOCK_DATA_TYPE
node.send(getdata.command, getdata.serialize())
# wait for the merkleblock command
envelope = node.wait_for_commands([b'merkleblock'])
print(envelope)
# wait for the tx command
envelope = node.wait_for_commands([b'tx'])
print(envelope)

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
sending: version: 7f11010000000000000000000dc0bf5b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d208d07ece835fe28291b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
receiving: version: 7f1101000d040000000000000ec0bf5b00000000000000000000000000000000000000000000ffff4830fd33cf700d04000000000000000000000000000000000000000000000000f555f8496387e698102f5361746f7368693a302e31372e302f79e9150001
sending: verack: 
receiving: verack: 
sending: filterload: 1e000000000000008800000002000000000000000000200000000100000000050000006260010001
sending: getdata: 0103000000f94f9dbeb59aaf899130199372baf9b223c052624386e916fd01000000000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: 48215b048bb8daf4
sending: pong: 48215b048bb8daf4
receiving: addr: 010ec0bf5b0d0400000000000000000000000000000000ffffa2d49eac479d
receiving

### Exercise 3

#### 3.1. Make [this test](/edit/session8/merkleblock.py) pass.

In [10]:
# Exercise 3.1

reload(merkleblock)
run_test(merkleblock.MerkleBlockTest('test_is_valid'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 4

#### 4.1. You have been sent some unknown amount of testnet bitcoins to your address. 

Send all of it back (minus fees) to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv` using only the networking protocol.

In [12]:
# Exercise 4.1

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import (
    decode_base58,
    double_sha256,
    encode_varint,
    little_endian_to_int, 
    read_varint,
    SIGHASH_ALL,
)
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script
from tx import Tx, TxIn, TxOut

last_block_hex = '0000000000000b8e8529996b47b8c3dd13c70adfaf94278883e05bba6ae9159a'
passphrase = b'jimmy@programmingblockchain.com secret passphrase'  # FILL THIS IN
secret = little_endian_to_int(double_sha256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
filter_size = 30
filter_num_functions = 5
filter_tweak = 90210  # FILL THIS IN

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis

# get the hash160 of the address, which is what we need to add to our filter
h160 = decode_base58(addr)
# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
bf = BloomFilter(filter_size, filter_num_functions, filter_tweak)
# add the h160 to the bloom filter
bf.add(h160)

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)
# complete the handshake
node.handshake()
# load the bloom filter with the filterload command
node.send(b'filterload', bf.filterload())

# set start block to last_block from above
start_block = bytes.fromhex(last_block_hex)
# create GetHeadersMessage with this start block
ghm = GetHeadersMessage(start_block=start_block)
# send a getheaders message with the start block
node.send(ghm.command, ghm.serialize())

# wait for the headers message
envelope = node.wait_for_commands([HeadersMessage.command])

# get the stream from the headers
s = envelope.stream()
# parse the headers message
headers_message = HeadersMessage.parse(s)
# store the last block as None
last_block = None
# initialize the GetDataMessage
getdata = GetDataMessage()
# loop through the blocks in the headers
for block in headers_message.blocks:
    # check that the proof of work on the block is valid
    if not block.check_pow():
        raise RuntimeError
    # check that this block's prev_block is the last block
    if last_block is not None and block.prev_block != last_block:
        raise RuntimeError
    # add FILTERED_BLOCK_DATA_TYPE block.hash() to GetDataMessage
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, block.hash())
    # set the last block to the current hash
    last_block = block.hash()
# send the getdata message
node.send(getdata.command, getdata.serialize())

# initialize prev_tx and prev_index to None
prev_tx, prev_index = None, None
# initialize prev_amount to None
prev_amount = None
# while prev_tx is None 
while prev_tx is None:
    # wait for the merkleblock or tx commands
    envelope = node.wait_for_commands([b'tx', b'merkleblock'])
    # initialize the stream from the envelope
    s = envelope.stream()
    # if we have the merkleblock command
    if envelope.command == b'merkleblock':
        # parse the MerkleBlock
        mb = MerkleBlock.parse(s)
        # check that the MerkleBlock is valid
        if not mb.is_valid():
            raise RuntimeError
    # else we have the tx command
    else:
        # parse the tx
        t = Tx.parse(s)
        # loop through the enumerated tx outs (enumerate(t.tx_outs))
        for i, tx_out in enumerate(t.tx_outs):
            print(tx_out)
            # if our output has the same address as our address (addr) we found it
            if tx_out.script_pubkey.address(testnet=True) == addr:
                # we found our utxo. set prev_tx, prev_index, prev_amount
                prev_tx = t.hash()
                prev_index = i
                prev_amount = tx_out.amount
                # set the cache so we don't look up the tx in a block explorer
                # Use: TxIn.set_cache(prev_tx, t.serialize())
                TxIn.set_cache(prev_tx, t.serialize())
# create tx_in
tx_in = TxIn(prev_tx, i, b'', 0xffffffff)
# calculate the output amount (prev_amount - fee)
output_amount = prev_amount - fee
# create a new TxOut to target_script.serialize() with the output amount
tx_out = TxOut(output_amount, target_script)
# create a new transaction (testnet=True)
t = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# sign the transaction at input 0 with your private key and SIGHASH_ALL
t.sign_input(0, private_key, SIGHASH_ALL)
# serialize and hex to see what it looks like
print(t.serialize().hex())
# send this signed transaction on the network
node.send(b'tx', t.serialize())
# wait a sec so this message goes through to the other node time.sleep(1) 
time.sleep(1)
# now ask for this transaction from the other node
# create a GetDataMessage
getdata = GetDataMessage()
# ask for our transaction by adding its hash and TX_DATATYPE to the message
getdata.add_data(TX_DATA_TYPE, t.hash())
# send the message
node.send(getdata.command, getdata.serialize())
# now wait for a response
envelope = node.wait_for_commands({b'tx', b'reject'})
if envelope.command == b'tx':
    print(envelope.payload.hex())
    print(t.hash().hex())
else:
    print(envelope.payload)

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
sending: version: 7f110100000000000000000018c0bf5b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d20c728f0203ebb2bd31b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
receiving: version: 7f1101000d0400000000000019c0bf5b00000000000000000000000000000000000000000000ffff4830fd33cf800d04000000000000000000000000000000000000000000000000177d83cab5388f1e102f5361746f7368693a302e31372e302f79e9150001
sending: verack: 
receiving: verack: 
sending: filterload: 1e000000000000008800000002000000000000000000200000000100000000050000006260010001
sending: getheaders: 7f110100019a15e96aba5be083882794afdf0ac713ddc3b8476b9929858e0b0000000000000000000000000000000000000000000000000000000000000000000000000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: efcaa67263a33e3a
sending: pong: efcaa67263a33e3a
receiving: addr: 0119c